# Lesson 15. SimPy &mdash; priority queues, preemption

### SA421 Fall 2015

## Priority queues

**Problem.** Dantzig's Diner is very, very small: it only has a 4-seat counter.

Customers arrive individually at the diner according to an exponential interarrival time distribution with a mean of 15 minutes. 25% of these customers are VIPs, and get to skip to the head of the queue upon arrival (behind any VIPs already in line, but ahead of any non-VIPs). The time that each customer spends at the diner is uniformly distributed between 60 and 90 minutes.

* Simulate the restaurant for 4 hours.


* What is the average delay of VIP customers? What is the average delay of non-VIP customers?

* Below, we have SimPy code for this problem, <span style="color:#a00000">under the assumption that all customers are non-VIPs</span>.


* Print statements have been added to output when a customer arrives, when a customer starts service, and when a customer finishes service.

In [ ]:
##### Setup #####
# Import everything from SimPy
from SimPy.Simulation import *

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, rand, exponential, uniform

In [ ]:
##### Parameters #####
class P:
    # Customers arrive at diner according to  
    # exponentially distributed interarrival times with mean 15
    interarrivalTimeMean = 15
    
    # Service times are uniformly distributed between 60 and 90 minutes
    serviceTimeMin = 60
    serviceTimeMax = 90
    
    # 4 seats at diner
    nSeats = 4
    
    # Simulate restaurant for 4 hours
    simulationTimeMax = 4 * 60
    
    
##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        print("Time {0}: {1} arrives".format(now(), self.name))
        yield request, self, R.seats  
        
        # Customer is released from queue and starts service
        print("Time {0}: {1} starts service".format(now(), self.name))
        serviceTime = uniform(low = P.serviceTimeMin, high = P.serviceTimeMax)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        print("Time {0}: {1} finishes service".format(now(), self.name))
        yield release, self, R.seats

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

            
##### Resources #####
class R:
    # Seats
    seats = None


##### Model #####
def model(inputSeed):
    # Initialize SimPy 
    initialize()

    # Initialize seed for random number generator
    seed(inputSeed)

    # Create the server resource
    R.seats = Resource(capacity = P.nSeats)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* First, let's create a function that generates random samples from the VIP/non-VIP customer distribution: let 0 correspond to a non-VIP, and let 1 correspond to a VIP.

In [ ]:
##### Additional Setup #####
# Import infinity from NumPy
from numpy import inf

# Import bisect_right from bisect
from bisect import bisect_left

In [ ]:
##### Distributions #####
class D:
    # VIP/non-VIP distribution
    # 0 = non-VIP, 1 = VIP
    def VIP():
        # List of possible values, including -inf
        a = 
        
        # cdf at values
        cdf = 
        
        # Generate variate
        u = rand()
        i = bisect_left(cdf, u)
        variate = a[i]

        # Return variate
        return variate

* We need to modify the above code so that the priority of customers is taken into account.


* This can be accomplished by adding a **priority value** to the `yield request` statement:
```
yield request, self, resourceName, priorityValue
```


* The priority value can be any number: processes with higher priority values will be placed closer to the beginning of the queue.


* In other words, the queue order is <span style="color:#a00000;">dynamic</span>: processes are always ordered in the queue according to their priority values.


* Let's implement the problem setting as follows:
    - Determine the customer type in the Customer process, right before he/she joins the queue.
         - We could have also done this using multiple customer processes, like in the previous lesson.
    - Non-VIPs will join the queue with priority 0.
    - VIPs will join the queue with priority 100.

In [ ]:
##### Modify Customer process to reflect VIP and non-VIP behavior #####
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        print("Time {0}: {1} arrives".format(now(), self.name))
        yield request, self, R.seats  
        
        # Customer is released from queue and starts service
        print("Time {0}: {1} starts service".format(now(), self.name))
        serviceTime = uniform(low = P.serviceTimeMin, high = P.serviceTimeMax)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        print("Time {0}: {1} finishes service".format(now(), self.name))
        yield release, self, R.seats

* To have a resource's queue respect the given priority values, we need to specify `qType = PriorityQ` when creating the resource with `Resource()`.

In [ ]:
##### Modify the model() function to activate priority queueing #####
def model(inputSeed):
    # Initialize SimPy 
    initialize()

    # Initialize seed for random number generator
    seed(inputSeed)

    # Create the server resource, activate priority queue
    R.seats = Resource(capacity = P.nSeats)

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* Let's run the model and see what happens.

In [ ]:
model(456)

* What's going on with Customer 9?

*Write your notes here. Double-click to edit.*

## With your neighbor...

* Implement two monitors:
    - `M.waitVIP` for the delay of each VIP customer
    - `M.waitNonVIP` for the delay of each non-VIP customer
    

* Use these monitors to compute the average delay for each type of customer.

In [ ]:
##### Declare dummy variables for new monitors #####
class M:
    

In [ ]:
##### Activate new monitors #####


In [ ]:
##### Change Customer process to record delay using newly activated monitors #####


In [ ]:
##### Compute average delay for each customer type #####


## Preemption

* Now suppose that upon arrival, VIPs can displace any non-VIP customer in service (what an unfriendly diner!).


* The displaced customer will resume service when all VIPs are gone. 


* In other words, service for VIPs **preempts** service for non-VIPs.


* Some more realistic examples of preemption...
    - Manufacturing: rush-order jobs
    - High-performance computing clusters: higher priority jobs
    - What else?


* This change is easy to accomplish: we only need to add the argument `preemptable = True` to the `Resource()` call:

In [ ]:
##### Simulation #####
def model(inputSeed):
    # Initialize SimPy 
    initialize()

    # Initialize seed for random number generator
    seed(inputSeed)

    # Create the server resource, activate priority queue
    R.seats = Resource(capacity = P.nSeats, qType = PriorityQ, preemptable = True)
    
    # Create monitor for VIP customer delay
    M.delayVIP = Monitor()
    
    # Create monitor for non-VIP customer delay
    M.delayNonVIP = Monitor()

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* Let's run the model again.

In [ ]:
model(456)

* Let's print the average delay for both type of customers.

In [ ]:
print("Average delay for VIP customers = {0}".format(M.delayVIP.mean()))
print("Average delay for non-VIP customers = {0}".format(M.delayNonVIP.mean()))

* Does this make sense?

*Write your notes here. Double-click to edit.*